In [ ]:
from __future__ import print_function, division

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np

class ACGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=losses,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid, target_label = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([noise, label], [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(512 * 4 * 4, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((4, 4, 512)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3,padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, 100)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.summary()

        img = Input(shape=self.img_shape)

        # Extract feature representation
        features = model(img)

        # Determine validity and label of the image
        validity = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes, activation="softmax")(features)

        return Model(img, [validity, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = cifar10.load_data()

        # Configure inputs
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            # idx = np.random.randint(0, X_train.shape[0], batch_size)
            for index in range(nb_batches):
                global_step += 1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                # Image labels. 0-9
                img_labels = y_train[index * batch_size:(index + 1) * batch_size]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # The labels of the digits that the generator tries to create an
                # image representation of
                sampled_labels = np.random.randint(0, 10, (batch_size, 1))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, sampled_labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(image_batch, [valid, img_labels])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, sampled_labels])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[3], 100 * d_loss[4], g_loss[0]))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images_new(epoch, global_step)


    def sample_images(self, epoch):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0])
                axs[i,j].axis('off')
                cnt += 1

        if not os.path.isdir('images_acgan_cifar10'):
            os.mkdir('images_acgan_cifar10')

        fig.savefig("images_acgan_cifar10/%d.png" % epoch)
        plt.close()

    def sample_images_new(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        generated_images = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        generated_images = np.asarray((generated_images * 127.5 + 127.5).astype(np.uint8))

        def vis_square(data, padsize=1, padval=0):
            # force the number of filters to be square
            n = int(np.ceil(np.sqrt(data.shape[0])))
            padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
            data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))

            # tile the filters into an image
            data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
            data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
            return data

        img = vis_square(generated_images)
        if not os.path.isdir('images_acgan_cifar10'):
            os.mkdir('images_acgan_cifar10')
        Image.fromarray(img).save(
            "images_acgan_cifar10/epoch_%d_step_%d.png" % (epoch,global_step))



if __name__ == '__main__':
    acgan = ACGAN()
    acgan.train(epochs=50, batch_size=64, sample_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 16, 16, 64)        1792      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 8, 8, 128)         73856     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 8, 8, 128)         0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 8, 8, 128)         512       
__________

epoch:0 step:41 [D loss: 2.819698, acc.: 89.84%, op_acc: 19.53%] [G loss: 7.897219]
epoch:0 step:42 [D loss: 3.438981, acc.: 49.22%, op_acc: 17.97%] [G loss: 11.185755]
epoch:0 step:43 [D loss: 2.893182, acc.: 82.81%, op_acc: 21.09%] [G loss: 10.761623]
epoch:0 step:44 [D loss: 3.177279, acc.: 75.78%, op_acc: 16.41%] [G loss: 8.748610]
epoch:0 step:45 [D loss: 2.932548, acc.: 85.94%, op_acc: 24.22%] [G loss: 8.013006]
epoch:0 step:46 [D loss: 3.308771, acc.: 53.91%, op_acc: 17.97%] [G loss: 9.694947]
epoch:0 step:47 [D loss: 3.078465, acc.: 86.72%, op_acc: 20.31%] [G loss: 9.524560]
epoch:0 step:48 [D loss: 2.931798, acc.: 80.47%, op_acc: 14.84%] [G loss: 7.220007]
epoch:0 step:49 [D loss: 3.470091, acc.: 47.66%, op_acc: 12.50%] [G loss: 10.636581]
epoch:0 step:50 [D loss: 2.832927, acc.: 89.06%, op_acc: 16.41%] [G loss: 9.863018]
epoch:0 step:51 [D loss: 3.069717, acc.: 85.16%, op_acc: 14.84%] [G loss: 8.132058]
epoch:0 step:52 [D loss: 3.068965, acc.: 68.75%, op_acc: 20.31%] [G loss:

epoch:0 step:141 [D loss: 2.601242, acc.: 97.66%, op_acc: 14.84%] [G loss: 6.548918]
epoch:0 step:142 [D loss: 2.539717, acc.: 98.44%, op_acc: 22.66%] [G loss: 7.564958]
epoch:0 step:143 [D loss: 2.940918, acc.: 77.34%, op_acc: 22.66%] [G loss: 10.331885]
epoch:0 step:144 [D loss: 2.350083, acc.: 97.66%, op_acc: 25.78%] [G loss: 12.326757]
epoch:0 step:145 [D loss: 2.391638, acc.: 96.09%, op_acc: 21.09%] [G loss: 9.514400]
epoch:0 step:146 [D loss: 2.685634, acc.: 94.53%, op_acc: 17.97%] [G loss: 6.722279]
epoch:0 step:147 [D loss: 2.344292, acc.: 97.66%, op_acc: 25.00%] [G loss: 6.847382]
epoch:0 step:148 [D loss: 2.615364, acc.: 94.53%, op_acc: 19.53%] [G loss: 6.956544]
epoch:0 step:149 [D loss: 2.511181, acc.: 97.66%, op_acc: 20.31%] [G loss: 6.967930]
epoch:0 step:150 [D loss: 2.472699, acc.: 97.66%, op_acc: 25.78%] [G loss: 7.163852]
epoch:0 step:151 [D loss: 2.428761, acc.: 97.66%, op_acc: 28.91%] [G loss: 7.368659]
epoch:0 step:152 [D loss: 2.478817, acc.: 94.53%, op_acc: 19.53

epoch:0 step:241 [D loss: 2.214991, acc.: 96.09%, op_acc: 30.47%] [G loss: 8.155568]
epoch:0 step:242 [D loss: 2.313529, acc.: 97.66%, op_acc: 25.78%] [G loss: 7.477774]
epoch:0 step:243 [D loss: 2.232189, acc.: 97.66%, op_acc: 28.12%] [G loss: 7.754216]
epoch:0 step:244 [D loss: 2.777728, acc.: 83.59%, op_acc: 21.88%] [G loss: 10.487707]
epoch:0 step:245 [D loss: 2.419351, acc.: 94.53%, op_acc: 25.00%] [G loss: 8.326757]
epoch:0 step:246 [D loss: 2.510192, acc.: 92.19%, op_acc: 27.34%] [G loss: 7.134171]
epoch:0 step:247 [D loss: 2.595766, acc.: 92.19%, op_acc: 25.00%] [G loss: 7.410471]
epoch:0 step:248 [D loss: 2.384381, acc.: 95.31%, op_acc: 23.44%] [G loss: 7.162591]
epoch:0 step:249 [D loss: 2.469911, acc.: 96.88%, op_acc: 23.44%] [G loss: 7.119973]
epoch:0 step:250 [D loss: 2.446281, acc.: 96.09%, op_acc: 24.22%] [G loss: 6.620314]
epoch:0 step:251 [D loss: 2.415916, acc.: 96.09%, op_acc: 24.22%] [G loss: 6.393055]
epoch:0 step:252 [D loss: 2.539431, acc.: 97.66%, op_acc: 25.00%

epoch:0 step:341 [D loss: 1.897848, acc.: 95.31%, op_acc: 31.25%] [G loss: 6.049740]
epoch:0 step:342 [D loss: 2.125501, acc.: 94.53%, op_acc: 24.22%] [G loss: 5.161163]
epoch:0 step:343 [D loss: 1.948997, acc.: 91.41%, op_acc: 33.59%] [G loss: 4.371329]
epoch:0 step:344 [D loss: 2.118573, acc.: 85.94%, op_acc: 31.25%] [G loss: 5.997491]
epoch:0 step:345 [D loss: 1.971422, acc.: 92.19%, op_acc: 32.81%] [G loss: 5.419975]
epoch:0 step:346 [D loss: 1.933678, acc.: 89.06%, op_acc: 34.38%] [G loss: 5.672556]
epoch:0 step:347 [D loss: 1.957331, acc.: 92.97%, op_acc: 36.72%] [G loss: 5.052114]
epoch:0 step:348 [D loss: 2.171093, acc.: 82.81%, op_acc: 34.38%] [G loss: 5.285692]
epoch:0 step:349 [D loss: 2.014226, acc.: 92.97%, op_acc: 32.81%] [G loss: 6.507783]
epoch:0 step:350 [D loss: 2.187095, acc.: 79.69%, op_acc: 35.16%] [G loss: 6.077156]
epoch:0 step:351 [D loss: 1.888190, acc.: 94.53%, op_acc: 34.38%] [G loss: 6.607055]
epoch:0 step:352 [D loss: 1.989724, acc.: 85.94%, op_acc: 44.53%]

...

epoch:48 step:37525 [D loss: 0.649964, acc.: 74.22%, op_acc: 94.53%] [G loss: 1.606641]
epoch:48 step:37526 [D loss: 0.921388, acc.: 71.88%, op_acc: 89.84%] [G loss: 2.013302]
epoch:48 step:37527 [D loss: 0.776032, acc.: 78.12%, op_acc: 90.62%] [G loss: 1.827012]
epoch:48 step:37528 [D loss: 0.874644, acc.: 67.19%, op_acc: 89.84%] [G loss: 1.767404]
epoch:48 step:37529 [D loss: 0.847171, acc.: 71.09%, op_acc: 89.06%] [G loss: 1.656095]
epoch:48 step:37530 [D loss: 0.655643, acc.: 78.12%, op_acc: 93.75%] [G loss: 1.869509]
epoch:48 step:37531 [D loss: 0.758286, acc.: 75.00%, op_acc: 91.41%] [G loss: 1.569540]
epoch:48 step:37532 [D loss: 0.796108, acc.: 73.44%, op_acc: 89.84%] [G loss: 1.865758]
epoch:48 step:37533 [D loss: 0.667705, acc.: 78.91%, op_acc: 92.97%] [G loss: 1.682251]
epoch:48 step:37534 [D loss: 0.724297, acc.: 79.69%, op_acc: 92.19%] [G loss: 2.131476]
epoch:48 step:37535 [D loss: 0.769737, acc.: 78.12%, op_acc: 88.28%] [G loss: 1.856116]
epoch:48 step:37536 [D loss: 0.9

epoch:48 step:37621 [D loss: 0.958750, acc.: 75.78%, op_acc: 81.25%] [G loss: 1.986867]
epoch:48 step:37622 [D loss: 0.842879, acc.: 75.00%, op_acc: 89.06%] [G loss: 1.731040]
epoch:48 step:37623 [D loss: 0.904977, acc.: 64.84%, op_acc: 89.06%] [G loss: 1.549159]
epoch:48 step:37624 [D loss: 0.896856, acc.: 74.22%, op_acc: 90.62%] [G loss: 1.642823]
epoch:48 step:37625 [D loss: 1.010594, acc.: 63.28%, op_acc: 84.38%] [G loss: 1.544329]
epoch:48 step:37626 [D loss: 0.673347, acc.: 78.12%, op_acc: 92.19%] [G loss: 1.999283]
epoch:48 step:37627 [D loss: 0.796742, acc.: 68.75%, op_acc: 92.19%] [G loss: 1.803063]
epoch:48 step:37628 [D loss: 0.758877, acc.: 83.59%, op_acc: 85.94%] [G loss: 1.906730]
epoch:48 step:37629 [D loss: 0.910343, acc.: 74.22%, op_acc: 88.28%] [G loss: 2.058123]
epoch:48 step:37630 [D loss: 0.736690, acc.: 77.34%, op_acc: 92.97%] [G loss: 1.784096]
epoch:48 step:37631 [D loss: 0.971904, acc.: 71.09%, op_acc: 90.62%] [G loss: 1.866379]
epoch:48 step:37632 [D loss: 0.8

epoch:48 step:37717 [D loss: 0.790621, acc.: 77.34%, op_acc: 91.41%] [G loss: 1.828934]
epoch:48 step:37718 [D loss: 0.849430, acc.: 75.78%, op_acc: 88.28%] [G loss: 1.836283]
epoch:48 step:37719 [D loss: 0.701178, acc.: 75.00%, op_acc: 89.84%] [G loss: 1.858760]
epoch:48 step:37720 [D loss: 0.740633, acc.: 75.78%, op_acc: 90.62%] [G loss: 1.840762]
epoch:48 step:37721 [D loss: 0.712097, acc.: 75.78%, op_acc: 93.75%] [G loss: 1.635717]
epoch:48 step:37722 [D loss: 0.839717, acc.: 68.75%, op_acc: 91.41%] [G loss: 1.687612]
epoch:48 step:37723 [D loss: 0.818864, acc.: 74.22%, op_acc: 89.06%] [G loss: 1.757330]
epoch:48 step:37724 [D loss: 0.773997, acc.: 82.81%, op_acc: 89.84%] [G loss: 1.701762]
epoch:48 step:37725 [D loss: 0.720322, acc.: 75.00%, op_acc: 92.19%] [G loss: 1.863826]
epoch:48 step:37726 [D loss: 0.769610, acc.: 78.91%, op_acc: 90.62%] [G loss: 1.994114]
epoch:48 step:37727 [D loss: 0.797396, acc.: 71.88%, op_acc: 88.28%] [G loss: 2.114992]
epoch:48 step:37728 [D loss: 0.8

epoch:48 step:37813 [D loss: 0.964217, acc.: 60.94%, op_acc: 90.62%] [G loss: 1.675641]
epoch:48 step:37814 [D loss: 0.825889, acc.: 63.28%, op_acc: 89.06%] [G loss: 1.866233]
epoch:48 step:37815 [D loss: 0.771945, acc.: 75.00%, op_acc: 92.97%] [G loss: 1.879002]
epoch:48 step:37816 [D loss: 0.824055, acc.: 70.31%, op_acc: 87.50%] [G loss: 1.963132]
epoch:48 step:37817 [D loss: 0.919016, acc.: 76.56%, op_acc: 89.84%] [G loss: 1.568085]
epoch:48 step:37818 [D loss: 0.746365, acc.: 70.31%, op_acc: 92.19%] [G loss: 1.478770]
epoch:48 step:37819 [D loss: 0.854553, acc.: 67.19%, op_acc: 86.72%] [G loss: 1.604906]
epoch:48 step:37820 [D loss: 0.832967, acc.: 74.22%, op_acc: 92.19%] [G loss: 1.652439]
epoch:48 step:37821 [D loss: 0.729661, acc.: 78.91%, op_acc: 92.19%] [G loss: 2.013949]
epoch:48 step:37822 [D loss: 0.850958, acc.: 69.53%, op_acc: 87.50%] [G loss: 1.625389]
epoch:48 step:37823 [D loss: 0.897787, acc.: 71.09%, op_acc: 89.84%] [G loss: 1.903216]
epoch:48 step:37824 [D loss: 0.8

epoch:48 step:37909 [D loss: 0.862344, acc.: 74.22%, op_acc: 89.84%] [G loss: 1.687057]
epoch:48 step:37910 [D loss: 0.924022, acc.: 68.75%, op_acc: 92.19%] [G loss: 1.466276]
epoch:48 step:37911 [D loss: 0.917377, acc.: 67.19%, op_acc: 89.06%] [G loss: 1.548863]
epoch:48 step:37912 [D loss: 0.780860, acc.: 75.00%, op_acc: 92.19%] [G loss: 1.441358]
epoch:48 step:37913 [D loss: 0.858646, acc.: 67.97%, op_acc: 90.62%] [G loss: 1.700255]
epoch:48 step:37914 [D loss: 0.800270, acc.: 80.47%, op_acc: 85.94%] [G loss: 1.714088]
epoch:48 step:37915 [D loss: 0.777187, acc.: 74.22%, op_acc: 88.28%] [G loss: 1.818392]
epoch:48 step:37916 [D loss: 0.952779, acc.: 64.84%, op_acc: 86.72%] [G loss: 1.653276]
epoch:48 step:37917 [D loss: 0.760167, acc.: 77.34%, op_acc: 92.19%] [G loss: 1.866941]
epoch:48 step:37918 [D loss: 0.883578, acc.: 67.97%, op_acc: 88.28%] [G loss: 1.644460]
epoch:48 step:37919 [D loss: 0.888196, acc.: 69.53%, op_acc: 90.62%] [G loss: 1.654061]
epoch:48 step:37920 [D loss: 0.7